# GROUP 5 MEMBERS - ICS 4A

136291 Shubh Patel

135736 Sheikh Ali

137223 Prachi Patel

135251 Evans Okania

136730 Kevin Kipkoech

116827 Kweyu Nicolas Anelka

136663 Sean Kimutai

134786 Ryan Lagat


# LL1 PARSER IMPLEMENTATION

## NOTE
The parser only works with grammar that does not have left recursion

----------------------------------------------------------------------------------------------------------------------------------------------------------------

In the below code, we have a function called calculate_first that is used to compute the first() set of non-terminal symbols in a given grammar. The first() set represents the set of terminal symbols that can appear as the first symbol of a derivation from a non-terminal symbol.

The code is structured as a function called calculate_first that takes in several parameters:

rule: The current rule for which we want to compute the first() set.

terminal_symbols: The set of terminal symbols in the grammar.

production_rules: A dictionary that contains the production rules for each non-terminal symbol.

first_sets: A dictionary that stores the computed first() sets for each non-terminal symbol.

The function uses recursion to compute the first() set for a given rule. It checks the first symbol of the rule and performs different actions based on its type (terminal, epsilon, or non-terminal).

In [103]:
#function to compute first() of the non-terminal symbols
def calculate_first(rule, terminal_symbols, production_rules, first_sets):
    # Check if the first symbol of the rule is a terminal
    if len(rule) != 0 and rule[0] in terminal_symbols:
        return rule[0]

    # Check if the first symbol is an epsilon (empty)
    elif len(rule) != 0 and rule[0] == '#':
        return '#'

    # Check if the first symbol is a non-terminal
    elif len(rule) != 0 and rule[0] in production_rules:
        first_result = []
        rhs_rules = production_rules[rule[0]]

        # Calculate the first() set for all alternatives in the rule
        for sub_rule in rhs_rules:
            individual_result = calculate_first(sub_rule, terminal_symbols, production_rules, first_sets)
            if isinstance(individual_result, list):
                first_result.extend(individual_result)
            else:
                first_result.append(individual_result)

        # Remove epsilon if present and consider the next symbol
        if '#' not in first_result:
            return first_result
        else:
            first_result.remove('#')
            if len(rule) > 1:
                # Calculate the first() set for the remaining symbols
                new_result = calculate_first(rule[1:], terminal_symbols, production_rules, first_sets)
                if new_result is not None:
                    if isinstance(new_result, list):
                        return first_result + new_result
                    else:
                        return first_result + [new_result]
                else:
                    return first_result
            else:
                first_result.append('#')
                return first_result
    else:
        return None

In the below code, we have a function called calculate_follow that computes the follow() set of non-terminal symbols in a given grammar. The follow() set of a non-terminal symbol consists of all the terminal symbols that can appear immediately after occurrences of the non-terminal symbol in the production rules of the grammar.

The code is structured as a function named calculate_follow(). It takes the following parameters:

non_terminal: The non-terminal symbol for which the follow() set is to be calculated.

start_symbol: The start symbol of the grammar.

production_rules: A dictionary that contains the production rules of the grammar.

follow_sets: A dictionary that stores the calculated follow() sets for each non-terminal symbol.

first_sets: A dictionary that stores the calculated first() sets for each non-terminal symbol.

terminals: A list of terminal symbols in the grammar.

The function returns a list containing the computed follow() set for the given non-terminal symbol.

In [104]:
#function to compute follow() of the non-terminal symbols
def calculate_follow(non_terminal, start_symbol, production_rules, follow_sets, first_sets, terminals):
    result_set = set()

    #checks if the non-terminal is a start symbol
    if non_terminal == start_symbol:
        result_set.add('$')

    #loops through all production rules while computing the follow for each of the non-terminals
    for current_non_terminal in production_rules:
        rhs = production_rules[current_non_terminal]
        for sub_rule in rhs:
            if non_terminal in sub_rule:
                while non_terminal in sub_rule:
                    index_nt = sub_rule.index(non_terminal)
                    sub_rule = sub_rule[index_nt + 1:]
                    if len(sub_rule) != 0:
                        result = calculate_first(sub_rule, terminals, production_rules, first_sets)
                        if '#' in result:
                            newList = []
                            result.remove('#')
                            new_result = calculate_follow(current_non_terminal, start_symbol, production_rules, follow_sets, first_sets, terminals)
                            if new_result is not None:
                                if isinstance(new_result, list):
                                    result.extend(new_result)
                                    newList = result.extend(new_result)
                                else:
                                    result = [result] + new_result

                            else:
                                result = result
                    else:
                        if non_terminal != current_non_terminal:
                            result = calculate_follow(current_non_terminal, start_symbol, production_rules, follow_sets, first_sets, terminals)
                    if result is not None:
                        if isinstance(result, list):
                            result_set.update(result)
                        else:
                            result_set.add(result)
    return list(result_set)

In the below code, we have a function called compute_first_sets that is used to generate the first sets for a given set of grammar rules. The first set of a non-terminal symbol is the set of terminal symbols that can appear as the first symbol in any string derived from that non-terminal.

The code is structured as follows:

The compute_first_sets function takes four parameters: grammar_rules, terminal_symbols, production_rules, and first_sets.

The function first parses the grammar_rules to extract the left-hand side (lhs) and right-hand side (rhs) of each rule. The rhs can have multiple alternatives separated by the '|' symbol.

The production rules are stored in a dictionary where the lhs is the key and the rhs alternatives are stored as a list of lists.
The function then displays the grammar rules in a formatted manner.

Next, the function iterates over each non-terminal symbol in the production rules and calculates its first set.

For each sub-rule of a non-terminal symbol, the function calls the calculate_first function to calculate the first set.

The result of the calculate_first function is added to the first set of the non-terminal symbol.

Finally, the first sets for each non-terminal symbol are stored in the first_sets dictionary.

In [105]:
#Function to genrate first set using the grammer rules(calls the function calculate_first())
def compute_first_sets(grammar_rules, terminal_symbols, production_rules, first_sets):
    # Parse the grammar rules
    for rule in grammar_rules:
        parts = rule.split("->")
        lhs = parts[0].strip()
        rhs = parts[1].strip()
        multi_rhs = rhs.split('|')
        for i in range(len(multi_rhs)):
            multi_rhs[i] = multi_rhs[i].strip()
            multi_rhs[i] = multi_rhs[i].split()
        production_rules[lhs] = multi_rhs


    # Display the grammar rules
    print("\nGrammar Rules: \n")
    formatted_rules = []
    for non_terminal, rules in production_rules.items():
        formatted_rules.append(f"{non_terminal} -> {' | '.join([' '.join(rule) for rule in rules])}")

    for formatted_rule in formatted_rules:
        print(formatted_rule)

    #Calculate the first sets for each non-terminal
    for non_terminal in production_rules:
        first_set = set()
        for sub_rule in production_rules.get(non_terminal):
            result = calculate_first(sub_rule, terminal_symbols, production_rules, first_sets)
            if result is not None:
                if isinstance(result, list):
                    first_set.update(result)
                else:
                    first_set.add(result)
        first_sets[non_terminal] = first_set


In the below code, we have a function called compute_follow_sets that is used to generate the follow() set of non-terminal symbols in a given grammar. The follow() set of a non-terminal symbol consists of all the terminal symbols that can appear immediately after occurrences of the non-terminal symbol in the production rules of the grammar.

The function takes the following parameters:

start_symbol: The start symbol of the grammar.

production_rules: A dictionary that maps each non-terminal symbol to its production rules.

follow_sets: A dictionary that will store the follow sets for each non-terminal symbol.

first_sets: A dictionary that contains the first sets for each symbol.

terminals: A set of terminal symbols.

The function iterates over each non-terminal symbol in the production_rules dictionary and calls the function called calculate_follow to calculate the follow set for that symbol. The result is then added to the follow_sets dictionary.

In [106]:
#Function to generate follow set using the grammer rules(calls the function calculate_follow())
def compute_follow_sets(start_symbol, production_rules, follow_sets, first_sets, terminals):
    for non_terminal in production_rules:
        result_set = set()
        result = calculate_follow(non_terminal, start_symbol, production_rules, follow_sets, first_sets, terminals)
        if result is not None:
            if isinstance(result, list):
                result_set.update(result)
            else:
                result_set.add(result)
        follow_sets[non_terminal] = result_set

In the below code, we have a function called create_parse_table that takes four parameters: production_rules, first_sets, follow_sets, and terminal_symbols.

It creates the LL(1) parsing table based on these inputs and returns the table, a boolean indicating whether the grammar is LL(1), and the list of terminal symbols.

The code is divided into the following sections:

Printing First Sets and Follow Sets Table: This section prints the first sets and follow sets in a tabular format for better visualization.

Creating the LL(1) Parsing Table: This section initializes the LL(1) parsing table as a 2D list and iterates over the production rules to populate the table. It also checks for conflicts in the grammar and sets the grammar_is_LL1 flag accordingly.

Displaying the LL(1) Parsing Table: This section prints the LL(1) parsing table in a tabular format for better visualization.

Returning the Parsing Table: Finally, the function returns the LL(1) parsing table, the grammar_is_LL1 flag, and the list of terminal symbols.

In [107]:
# Function to create the LL(1) parsing table
def create_parse_table(production_rules, first_sets, follow_sets, terminal_symbols):
    import copy
    print("\nFirst Sets and Follow Sets Table\n")
    max_len_first = max(len(str(first_sets[nt])) for nt in production_rules)
    max_len_follow = max(len(str(follow_sets[nt])) for nt in production_rules)
    print(f"{{:<{10}}} {{:<{max_len_first + 5}}} {{:<{max_len_follow + 5}}}"
          .format("Non-Terminal", "FIRST", "FOLLOW"))
    for nt in production_rules:
        print(f"{{:<{10}}} {{:<{max_len_first + 5}}} {{:<{max_len_follow + 5}}}"
              .format(nt, str(first_sets[nt]), str(follow_sets[nt])))


    # Create the LL(1) parsing table
    non_terminal_list = list(production_rules.keys())
    terminals = copy.deepcopy(terminal_symbols)
    terminals.append('$')

    parse_table = [['' for _ in range(len(terminals))] for _ in range(len(production_rules))]

    grammar_is_LL1 = True

    for lhs in production_rules:
        rhs = production_rules[lhs]
        for sub_rule in rhs:
            result = calculate_first(sub_rule, terminal_symbols, production_rules, first_sets)
            if '#' in result:
                if isinstance(result, str):
                    first_follow = []
                    follow_op = follow_sets[lhs]
                    if isinstance(follow_op, str):
                        first_follow.append(follow_op)
                    else:
                        first_follow.extend(follow_op)
                    result = first_follow
                else:
                    result.remove('#')
                    result.extend(follow_sets[lhs])
            temp_list = []
            if isinstance(result, str):
                temp_list.append(result)
                result = copy.deepcopy(temp_list)
            for c in result:
                non_terminal_index = non_terminal_list.index(lhs)
                terminal_index = terminals.index(c)
                if parse_table[non_terminal_index][terminal_index] == '':
                    parse_table[non_terminal_index][terminal_index] = f"{lhs}->{' '.join(sub_rule)}"
                else:
                    if f"{lhs} -> {' '.join(sub_rule)}" not in parse_table[non_terminal_index][terminal_index]:
                        grammar_is_LL1 = False
                        parse_table[non_terminal_index][terminal_index] += f",{lhs}->{' '.join(sub_rule)}"

    # Display the LL(1) parsing table
    print("\nParse Table:\n")
    format_str = "{:>12}" * len(terminals)
    print(format_str.format(*terminals))

    for j in range(len(parse_table)):
        format_str1 = "{:>12}" * len(parse_table[j])
        print(f"{non_terminal_list[j]} {format_str1.format(*parse_table[j])}")

    return parse_table, grammar_is_LL1, terminals




In the below code we have a function called validate_input_string(), which takes the LL(1) parsing table, grammar information, terminal list, input string, terminal symbols, and start symbol as input parameters.

Input Validation: The function first checks if the grammar is LL(1) by evaluating the is_grammar_ll1 parameter. If the grammar is not LL(1), the function returns a message indicating that the input string cannot be validated.

Initialization: The function initializes the stack and buffer with the start symbol and the end-of-input marker ('$'), respectively. The buffer is created by splitting the input string into a list of terminal symbols in reverse order.

Parsing Loop: The function enters a while loop that continues until the stack and buffer are both empty. Within the loop, the function performs the following steps:

a. Check for Validity: If both the stack and buffer are empty, the function prints a message indicating that the input string is valid and returns.

b. Non-Terminal Processing: If the top of the stack is a non-terminal symbol, the function retrieves the corresponding entry from the parsing table based on the current non-terminal symbol and the next terminal symbol in the buffer. If the entry is not empty, the function applies the production rule by replacing the non-terminal symbol on the stack with the right-hand side of the production rule.

c. Terminal Matching: If the top of the stack is a terminal symbol, the function checks if it matches the next terminal symbol in the buffer. If there is a match, the function removes the matched terminal symbol from the buffer and the stack.

d. Invalid String: If none of the above conditions are met, the function returns a message indicating that the input string is invalid.

In [108]:
# Function to validate an input string using the LL(1) parsing table
def validate_input_string(parsing_table, is_grammar_ll1, terminal_list, input_string, terminal_symbols, start_symbol):
    print(f"\nValidate Input String: {input_string}\n")
    if not is_grammar_ll1:
        return f"\nInput String: \"{input_string}\"\nGrammar is not LL(1)"
    stack = [start_symbol, '$']
    buffer = ['$'] + input_string.split()[::-1]
    print("{:>20} {:>20} {:>20}".format("Buffer", "Stack", "Action"))
    while True:
        if stack == ['$'] and buffer == ['$']:
            print("{:>20} {:>20} {:>20}".format(' '.join(buffer), ' '.join(stack), "Valid"))
            return "\nValid String!"
        elif stack[0] not in terminal_symbols:
            non_terminal_index = list(production_rules.keys()).index(stack[0])
            terminal_index = terminal_list.index(buffer[-1])
            if parsing_table[non_terminal_index][terminal_index] != '':
                entry = parsing_table[non_terminal_index][terminal_index]
                print("{:>20} {:>20} {:>25}".format(' '.join(buffer), ' '.join(stack), f"T[{stack[0]}][{buffer[-1]}] = {entry}"))
                lhs_rhs = entry.split("->")
                lhs_rhs[1] = lhs_rhs[1].replace('#', '').strip()
                entry_rhs = lhs_rhs[1].split()
                stack = entry_rhs + stack[1:]
            else:
                return f"\nInvalid String! No rule at Table[{stack[0]}][{buffer[-1]}]."
        else:
            if stack[0] == buffer[-1]:
                print("{:>20} {:>20} {:>20}".format(' '.join(buffer), ' '.join(stack), f"Matched: {stack[0]}"))
                buffer.pop()
                stack.pop(0)
            else:
                return "\nInvalid String! Unmatched terminal symbols"


The below code is the main code for parsing LL(1) grammar rules.

Input grammar rules: The user is prompted to input the grammar rules in a specific format. Each rule should be entered on a separate line, using "|" for alternatives and "->" for production.

Input non-terminals and terminals: The user is prompted to input the non-terminals and terminals separated by spaces.

Input a sample input string: The user is prompted to input a sample input string for validation.

Compute first sets: The compute_first_sets function is called to calculate the first sets for the non-terminals.

Determine the start symbol: The start symbol is determined as the first non-terminal entered.

Compute follow sets: The compute_follow_sets function is called to calculate the follow sets for the non-terminals.

Create the LL(1) parsing table: The create_parse_table function is called to create the LL(1) parsing table and check if the grammar is LL(1).

Validate the sample input string: The validate_input_string function is called to validate the sample input string using the LL(1) parsing table.

In [109]:
# MAIN CODE - To call the above functions

# Input grammar rules
print("Input the grammar rules (one rule per line, use | for alternatives, and -> for production):")
grammar_rules = []
while True:
    rule_input = input("Enter a grammar rule or press Enter to finish: ")
    if rule_input == "":
        break
    grammar_rules.append(rule_input)


# Input non-terminals and terminals
print("Enter non-terminals separated by space:")
non_terminals = input().split()

print("Enter terminals separated by space:")
terminals = input().split()


# Input a sample input string for validation
sample_input_string = input("Enter a sample input string: ")


# Initialize dictionaries to store production rules, first() sets, and follow() sets
production_rules = {}
first_sets = {}
follow_sets = {}


compute_first_sets(grammar_rules, terminals, production_rules, first_sets)  # Calculate first() sets for non-terminals
start_symbol = non_terminals[0]  # Determine the start symbol
compute_follow_sets(start_symbol, production_rules, follow_sets, first_sets, terminals) # Calculate follow() sets for non-terminals

# Create the LL(1) parsing table and check if the grammar is LL(1)
parsing_table, is_grammar_LL1, terminal_list = create_parse_table(production_rules, first_sets, follow_sets, terminals)

# Validate the sample input string using the LL(1) parsing table
if sample_input_string:
    validation_result = validate_input_string(parsing_table, is_grammar_LL1, terminal_list, sample_input_string, terminals, start_symbol)
    print(validation_result)
else:
    print("\nNo input string provided")

# SAMPLE GRAMMAR TO INPUT

# GRAMMAR 1 - Example used in class

# Production Rules = [
#    "E -> T E'",
# 	 "E' -> + T E' | #",
# 	 "T -> F T'",
# 	 "T' -> * F T' | #",
# 	 "F -> ( E ) | id"
# ]
# Non Terminals = ['E','E\'','T','T\'','F']
# Terminals = ['(',')','id','+','*']
# Sample Input String ="id + id * id"
# RESULT = VALID STRING and grammar is LL(1)

# GRAMMAR 2

# Production Rules = [
#    "S -> X Y Z | Z",
#    "X -> m | n Y | #",
#    "Y -> q | #",
#    "Z -> o"
# ]
# Non Terminals = ['X', 'S', 'Y', 'Z']
# Terminals = ['m', 'o', 'n', 'q']
# Sample Input String = "n q q o"
# RESULT: Grammar is not LL(1)

# GRAMMAR 3

# Production Rules = [
#    "S -> a A b",
#    "A -> c | d"
# ]

# Non Terminals = ['S', 'A']
# Temrinals = ['a', 'b', 'c', 'd']
# Sample Input String = "a c b"
# RESULT = VALID STRING and grammar is LL(1)

Input the grammar rules (one rule per line, use | for alternatives, and -> for production):
Enter a grammar rule or press Enter to finish: E -> T E'
Enter a grammar rule or press Enter to finish: E' -> + T E' | #
Enter a grammar rule or press Enter to finish: T -> F T'
Enter a grammar rule or press Enter to finish: T' -> * F T' | #
Enter a grammar rule or press Enter to finish: F -> ( E ) | id
Enter a grammar rule or press Enter to finish: 
Enter non-terminals separated by space:
E E' T T' F
Enter terminals separated by space:
( ) id + *
Enter a sample input string: id + id * id

Grammar Rules: 

E -> T E'
E' -> + T E' | #
T -> F T'
T' -> * F T' | #
F -> ( E ) | id

First Sets and Follow Sets Table

Non-Terminal FIRST            FOLLOW                   
E          {'id', '('}      {'$', ')'}               
E'         {'+', '#'}       {'$', ')'}               
T          {'id', '('}      {'$', '+', ')'}          
T'         {'*', '#'}       {'$', '+', ')'}          
F          {'id', '